In [1]:
#  strategy="html"
from langchain_community.document_loaders.llmsherpa import LLMSherpaFileLoader
loader = LLMSherpaFileLoader(
    file_path="/home/ubuntu/T_RAG/testing/2407.13734v1.pdf",
    new_indent_parser=True,
    apply_ocr=True,
    strategy="html",
    llmsherpa_api_url="http://localhost:5010/api/parseDocument?renderFormat=all",
)
html_docs = loader.load()

In [4]:
html_docs

[Document(metadata={'source': '/home/ubuntu/T_RAG/testing/2407.13734v1.pdf'}, page_content='<html><h1>Understanding      Reinforcement      Learning-Based</h1><h1>Fine-Tuning      of      Diffusion      Models:      A      Tutorial      and      Review</h1><p>Masatoshi      Uehara*!,      Yulai      Zhao\\\\\\\'*,      Tommaso      Biancalani!,      and      Sergey      Levine?</p><p>‘Genentech</p><li>*Princeton      University</li><p>>University      of      California,      Berkeley</p><h2>July      19,      2024</h2><h2>Abstract</h2><p> \n \n  This      tutorial      provides   \n \na      comprehensive      survey      of      methods      for      fine-tuning      diffusion      models  \n \n  to      optimize      downstream      reward      functions.\nWhile      diffusion      models      are      widely      known      to      provide  \n \n  excellent      generative      modeling      capability,      practical      applications      in      domains      such      as      bi

In [8]:
#     strategy="sections"
from langchain_community.document_loaders.llmsherpa import LLMSherpaFileLoader
loader = LLMSherpaFileLoader(
    file_path="https://arxiv.org/pdf/2402.14207.pdf",
    new_indent_parser=True,
    apply_ocr=True,
    strategy="sections",
    llmsherpa_api_url="http://localhost:5010/api/parseDocument?renderFormat=all",
)
docs = loader.load()

In [15]:
docs

[Document(metadata={'source': 'https://arxiv.org/pdf/2402.14207.pdf', 'section_number': 0, 'section_title': 'Abstract'}, page_content='Abstract\n \n \n  We      study      how      to      apply      large      language      models  \n \n  to      write      grounded      and      organized      long-form      ar-  \n \n  ticles      from      scratch,      with      comparable      breadth  \n \n  and      depth      to      Wikipedia      pages.\nThis      underex-  \n \n  plored      problem      poses      new      challenges      at      the  \n \n  pre-writing      stage,      including      how      to      research  \n \n  the      topic      and      prepare      an      outline      prior      to      writ-  \n \n  ing.\nWe      propose      STORM,   \n \na      writing      system  \n \n  for      the      Synthesis      of      Topic      Outlines      through  \n \n  Retrieval      and      Multi-perspective      Question      Ask-  \n \n  ing.\nSTORM      models      the 

In [11]:
docs[0]

Document(metadata={'source': 'https://arxiv.org/pdf/2402.14207.pdf', 'section_number': 0, 'section_title': 'Abstract'}, page_content='Abstract\n \n \n  We      study      how      to      apply      large      language      models  \n \n  to      write      grounded      and      organized      long-form      ar-  \n \n  ticles      from      scratch,      with      comparable      breadth  \n \n  and      depth      to      Wikipedia      pages.\nThis      underex-  \n \n  plored      problem      poses      new      challenges      at      the  \n \n  pre-writing      stage,      including      how      to      research  \n \n  the      topic      and      prepare      an      outline      prior      to      writ-  \n \n  ing.\nWe      propose      STORM,   \n \na      writing      system  \n \n  for      the      Synthesis      of      Topic      Outlines      through  \n \n  Retrieval      and      Multi-perspective      Question      Ask-  \n \n  ing.\nSTORM      models      the  

In [13]:
#     strategy="sections"
from langchain_community.document_loaders.llmsherpa import LLMSherpaFileLoader
loader = LLMSherpaFileLoader(
    file_path="https://arxiv.org/pdf/2402.14207.pdf",
    new_indent_parser=True,
    apply_ocr=True,
    strategy="chunks",
    llmsherpa_api_url="http://localhost:5010/api/parseDocument?renderFormat=all",
)
chunks = loader.load()

In [14]:
chunks

[Document(metadata={'source': 'https://arxiv.org/pdf/2402.14207.pdf', 'chunk_number': 0, 'chunk_type': 'para'}, page_content='Abstract\n \n \n  We      study      how      to      apply      large      language      models  \n \n  to      write      grounded      and      organized      long-form      ar-  \n \n  ticles      from      scratch,      with      comparable      breadth  \n \n  and      depth      to      Wikipedia      pages.\nThis      underex-  \n \n  plored      problem      poses      new      challenges      at      the  \n \n  pre-writing      stage,      including      how      to      research  \n \n  the      topic      and      prepare      an      outline      prior      to      writ-  \n \n  ing.\nWe      propose      STORM,   \n \na      writing      system  \n \n  for      the      Synthesis      of      Topic      Outlines      through  \n \n  Retrieval      and      Multi-perspective      Question      Ask-  \n \n  ing.\nSTORM      models      the      pre-

In [17]:
class Document:
    def __init__(self, metadata, page_content):
        self.metadata = metadata
        self.page_content = page_content

# List of Document objects
documents =chunks

# Convert to list of dictionaries
documents_dicts = []
for doc in documents:
    doc_dict = {
        'metadata': doc.metadata,
        'page_content': doc.page_content
    }
    documents_dicts.append(doc_dict)

# Print the result
print(documents_dicts)


[{'metadata': {'source': 'https://arxiv.org/pdf/2402.14207.pdf', 'chunk_number': 0, 'chunk_type': 'para'}, 'page_content': 'Abstract\n \n \n  We      study      how      to      apply      large      language      models  \n \n  to      write      grounded      and      organized      long-form      ar-  \n \n  ticles      from      scratch,      with      comparable      breadth  \n \n  and      depth      to      Wikipedia      pages.\nThis      underex-  \n \n  plored      problem      poses      new      challenges      at      the  \n \n  pre-writing      stage,      including      how      to      research  \n \n  the      topic      and      prepare      an      outline      prior      to      writ-  \n \n  ing.\nWe      propose      STORM,   \n \na      writing      system  \n \n  for      the      Synthesis      of      Topic      Outlines      through  \n \n  Retrieval      and      Multi-perspective      Question      Ask-  \n \n  ing.\nSTORM      models      the      pre-wr

In [18]:
len(documents_dicts)

250

In [22]:
documents_dicts[1]['page_content']

'Abstract\n \n \n  For      evaluation,      we      curate      FreshWiki,   \n \na      dataset  \n \n  of      recent      high-quality      Wikipedia      articles,      and       formulate      outline      assessments      to      evaluate      the  \n \n  pre-writing      stage.\nWe      further      gather      feedback  \n \n  from      experienced      Wikipedia      editors.\nCom-  \n \n  pared      to      articles      generated      by      an      outline-  \n \n  driven      retrieval-augmented      baseline,      more      of  \n \n  STORM’;      articles      are      deemed      to      be      organized  \n \n  (by   \n \na      25%      absolute      increase)      and      broad      in      cov-  \n \n  erage      (by      10%).\nThe      expert      feedback      also  \n \n  helps      identify      new      challenges      for      generating  \n \n  grounded      long      articles,      such      as      source      bias  \n \n  transfer      and      over-a